In [1]:
import pandas as pd
import numpy as np
import requests
import json
import zipfile
import xmltodict
import time
import warnings
import re
import xmltodict
from tqdm import tqdm
from io import BytesIO

warnings.filterwarnings(action='ignore')

## Data Load

### [1] DART

In [2]:
# 1. 고유번호 -> 기업개황에 해당 정보 모두 포함. 활용 필요 X
data_dart_1 = pd.read_pickle('data_dart_1.pkl')
data_dart_1

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630
...,...,...,...,...
89361,01560457,삼원개발,None,20210506
89362,01556533,에이더블유파트너스,None,20210506
89363,01546299,동영와이케이,None,20210506
89364,00694605,미디어윌,None,20210506


In [3]:
# 2. 기업개황
data_dart_2 = pd.read_pickle('data_dart_2.pkl')
data_dart_2

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,,02-3668-0114,02-3668-0112,58221,19970722,12
0,000,정상,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,,(02) 3453-7111,(02) 3453-7345,58221,19981030,12
0,000,정상,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,,02-565-4510 (220),02-3453-4552,58221,19960410,12
0,000,정상,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,,02-518-0101 (변광섭),02-518-0133,58221,19950614,12
0,000,정상,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,,032-814-0981-4,032-814-0985,29294,19950509,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,000,정상,00521390,(주)뉴파워프라즈마,"New Power Plasma Co.,Ltd",뉴파워프라즈마,144960,위순임,K,1101111784952,2128144110,경기도 수원시 권선구 산업로155번길 214 (주)뉴파워프라즈마,newpower.co.kr,,031-612-7600,031-612-7699,29271,19991007,12
0,000,정상,00664853,제이씨케미칼(주),"JC Chemical Corporation, LTD.",제이씨케미칼,137950,"윤사호, 이종응",K,1358110124726,1358185115,울산광역시 울주군 온산읍 화산1길 70,www.jcchemical.co.kr,,02-6321-1333,027615404,2049,20060324,12
0,000,정상,00307028,경남제약 주식회사,"KYUNG NAM PHARM.CO.,LTD.",경남제약,053950,오성원,K,1350110085255,1348142258,경상남도 의령군 동동리 1539-5,www.kyungnampharm.co.kr,,02-3490-5105,02)3490-5129,212,19981229,12
0,000,정상,00874195,(주)코썬바이오,"Kosun Bio Co., Ltd.",코썬바이오,204990,조규면,E,1101113505421,2148794321,서울특별시 영등포구 선유로 146 이앤씨드림타워 207호,www.hsvital.com,,02-2628-0550,02-2628-0561,10797,20060808,12


In [4]:
# 3. 직원현황
data_dart_3 = pd.read_pickle('data_dart_3_rev.pkl')
data_dart_3

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,...,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,year,fyer_salary_totamt_rev,jan_salary_am_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,avrg_cnwk_sdytrn_rev
0,20200424000043,K,00956028,엑세스바이오,단위:USD,남,사무,12,NaN,NaN,...,12,3.36,"865,006","72,084",2020,8.650060e+05,72084.0,1.038007e+09,86500800.0,3.3
1,20200424000043,K,00956028,엑세스바이오,단위:USD,여,사무,8,NaN,NaN,...,8,3.81,"395,183","49,398",2020,3.951830e+05,49398.0,4.742196e+08,59277600.0,3.8
2,20200424000043,K,00956028,엑세스바이오,단위:USD,남,영업,4,NaN,NaN,...,4,4.18,"308,347","77,087",2020,3.083470e+05,77087.0,3.700164e+08,92504400.0,4.1
3,20200424000043,K,00956028,엑세스바이오,단위:USD,남,연구,7,NaN,NaN,...,7,3.67,"619,224","88,461",2020,6.192240e+05,88461.0,7.430688e+08,106153200.0,3.6
4,20200424000043,K,00956028,엑세스바이오,단위:USD,여,연구,3,NaN,NaN,...,3,4.57,"180,408","60,136",2020,1.804080e+05,60136.0,2.164896e+08,72163200.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16122,20210331003230,K,01264438,자이언트스텝,NaN,여,제작,68,NaN,NaN,...,68,2.3,"1,953,000,000","32,000,000",2021,1.953000e+09,32000000.0,1.953000e+09,32000000.0,2.0
16123,20210331003230,K,01264438,자이언트스텝,NaN,남,연구개발,14,NaN,NaN,...,14,1.8,"712,000,000","56,000,000",2021,7.120000e+08,56000000.0,7.120000e+08,56000000.0,1.0
16124,20210331003230,K,01264438,자이언트스텝,NaN,여,연구개발,6,NaN,NaN,...,6,1.1,"146,000,000","30,000,000",2021,1.460000e+08,30000000.0,1.460000e+08,30000000.0,1.0
16125,20210331003230,K,01264438,자이언트스텝,NaN,남,관리,11,2,NaN,...,13,4.1,"451,000,000","47,000,000",2021,4.510000e+08,47000000.0,4.510000e+08,47000000.0,4.0


In [5]:
# 4. 주요계정
data_dart_4 = pd.read_pickle('data_dart_5_rev.pkl') 
data_dart_4

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,nm,dt,amount
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,103213745
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,58222516
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,52946039
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,40517092
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,92215510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45725,20210329001000,11011,2020,00140566,002680,OFS,재무제표,IS,손익계산서,당기순이익,제 60 기,2018.01.01 ~ 2018.12.31,-3723580588
45726,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,매출액,제 8 기,2018.01.01 ~ 2018.12.31,276496903
45727,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,영업이익,제 8 기,2018.01.01 ~ 2018.12.31,-3375624884
45728,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 8 기,2018.01.01 ~ 2018.12.31,-7344671390


### [2] 국민연금

In [6]:
# 1. 사업장 정보조회 서비스
data_ins_1 = pd.read_pickle('data_ins_1.pkl')
data_ins_1

,bzowrRgstNo,dataCrtYm,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,seq,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,135813****,202104,41,550,360,32344013,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
1,653880****,202105,11,440,123,33251682,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
2,114871****,202103,11,680,107,31966734,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
3,410814****,202112,29,170,129,36603791,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
4,107811****,202112,47,111,111,36654621,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...
1981,111843****,202112,11,680,108,36902629,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
1982,318810****,201511,43,730,250,20310715,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
1983,433880****,202110,48,310,370,35940676,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
1984,139810****,202112,11,545,101,36501345,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1


In [7]:
# 2. 상세 정보조회
data_ins_2 = pd.read_pickle('data_ins_2.pkl')
data_ins_2

,adptDt,bzowrRgstNo,crrmmNtcAmt,jnngpCnt,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,scsnDt,vldtVlKrnNm,wkplIntpCd,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd
0,20020401,135813****,14996040,48,41,550,360,00010101,플라스틱 창호 제조업,252400,1,나노캠텍(주),경기도 안성시 양성면 양성로,1
0,20210104,653880****,909980,3,11,440,123,00010101,의료용 기구 소매업,523120,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1
0,20140625,114871****,4803720,13,11,680,107,00010101,사업 및 무형 재산권 중개업,749903,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1
0,19991125,410814****,83023960,301,29,170,129,00010101,주형 및 금형 제조업,292903,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1
0,19880101,107811****,95653940,210,47,111,111,00010101,그 외 기타 1차 철강 제조업,271902,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20190604,111843****,979380,3,11,680,108,00010101,응용 소프트웨어 개발 및 공급업,722000,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1
0,20140410,318810****,1089680,7,43,730,250,00010101,BIZ_NO미존재사업장,999999,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1
0,20210426,433880****,405000,3,48,310,370,00010101,"미장, 타일 및 방수 공사업",452101,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1
0,19880101,139810****,3945120,17,11,545,101,00010101,의약품 도매업,513311,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1


In [8]:
# 법정동 코드 데이터

data_dong_rev = pd.read_pickle('data_dong_rev.pkl')
data_dong_rev

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
2615,2600000000,부산광역시,존재
2912,2700000000,대구광역시,존재
3280,2800000000,인천광역시,존재
3624,2900000000,광주광역시,존재
3870,3000000000,대전광역시,존재
4054,3100000000,울산광역시,존재
4302,3611000000,세종특별자치시,존재
4456,4100000000,경기도,존재
10993,4200000000,강원도,존재


## Data Merge
* key값: [DART] -> corp_code, [국민연금] -> wkplNm, [DART&국민연금] -> 기업명 작업 후 매칭

In [9]:
# merge할 형태로 추가 정제
# 기업개황
data_dart_2_m = data_dart_2.loc[:, ['corp_code', 'corp_name', 'corp_name_eng', 'stock_name', 'stock_code',
                                   'ceo_nm', 'corp_cls', 'jurir_no', 'bizr_no', 'adres', 'hm_url', 'phn_no',
                                   'fax_no', 'induty_code', 'est_dt']]
data_dart_2_m['crop_cls_kr'] = data_dart_2_m.corp_cls.map({'Y': '유가', 'K': '코스닥', 'N': '코넥스', 'E': '기타'})
data_dart_2_m.reset_index(inplace=True, drop=True)
data_dart_2_m

,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,phn_no,fax_no,induty_code,est_dt,crop_cls_kr
0,00260985,한빛네트,"HanbitNet, Inc.",한빛네트,036720,강승환,E,1101111443079,2088124283,서울특별시 종로구 익선동 30-6,www.hanbitnet.com,02-3668-0114,02-3668-0112,58221,19970722,기타
1,00264529,(주)엔플렉스,"Nplex,Inc.",엔플렉스,040130,이 정 학,E,1101111601750,2208156354,서울특별시 강남구 논현동 98 동익빌딩 2층,www.gamekorea.net,(02) 3453-7111,(02) 3453-7345,58221,19981030,기타
2,00358545,(주)동서정보기술,"Dong Seo Information Technology Co., Ltd.",동서정보기술,055000,이희봉,E,1101111259525,2208103653,서울특별시 강남구 대치동 997-4,http://www.dsit.co.kr,02-565-4510 (220),02-3453-4552,58221,19960410,기타
3,00231567,주식회사 애드모바일,,애드모바일,032600,김용석,E,1101111166837,1208151156,서울특별시 강서구 가양동 1458-9 동남빌딩 2층,www.kdline.co.kr,02-518-0101 (변광섭),02-518-0133,58221,19950614,기타
4,00247939,(주)씨모스,"creative master of semiconductor. co.,ltd",씨모스,037600,유인엽,E,1201110118077,1398120545,인천광역시 남동구 고잔동 685-6,www.cmos21.co.kr,032-814-0981-4,032-814-0985,29294,19950509,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3396,00521390,(주)뉴파워프라즈마,"New Power Plasma Co.,Ltd",뉴파워프라즈마,144960,위순임,K,1101111784952,2128144110,경기도 수원시 권선구 산업로155번길 214 (주)뉴파워프라즈마,newpower.co.kr,031-612-7600,031-612-7699,29271,19991007,코스닥
3397,00664853,제이씨케미칼(주),"JC Chemical Corporation, LTD.",제이씨케미칼,137950,"윤사호, 이종응",K,1358110124726,1358185115,울산광역시 울주군 온산읍 화산1길 70,www.jcchemical.co.kr,02-6321-1333,027615404,2049,20060324,코스닥
3398,00307028,경남제약 주식회사,"KYUNG NAM PHARM.CO.,LTD.",경남제약,053950,오성원,K,1350110085255,1348142258,경상남도 의령군 동동리 1539-5,www.kyungnampharm.co.kr,02-3490-5105,02)3490-5129,212,19981229,코스닥
3399,00874195,(주)코썬바이오,"Kosun Bio Co., Ltd.",코썬바이오,204990,조규면,E,1101113505421,2148794321,서울특별시 영등포구 선유로 146 이앤씨드림타워 207호,www.hsvital.com,02-2628-0550,02-2628-0561,10797,20060808,기타


In [10]:
# type cast
data_dart_3['rgllbr_co'] = data_dart_3.rgllbr_co.fillna('0').str.replace(',', '').astype(int)
data_dart_3['rgllbr_abacpt_labrr_co'] = data_dart_3.rgllbr_abacpt_labrr_co.fillna('0').str.replace(',', '').astype(int)
data_dart_3['cnttk_co'] = data_dart_3.cnttk_co.fillna('0').str.replace(',', '').astype(int)
data_dart_3['cnttk_abacpt_labrr_co'] = data_dart_3.cnttk_abacpt_labrr_co.fillna('0').str.replace(',', '').astype(int)
data_dart_3['sm'] = data_dart_3.sm.fillna('0').str.replace(',', '').astype(int)

In [11]:
# merge할 형태로 추가 정제
# 직원현황

data_dart_3_m = data_dart_3.pivot_table(index=['corp_code', 'year', 'fo_bbm', 'sexdstn'],
                        #columns=['fo_bbm'],
                        aggfunc={'rgllbr_co': 'sum', 'rgllbr_abacpt_labrr_co': 'sum', 
                                 'cnttk_co': 'sum', 'cnttk_abacpt_labrr_co': 'sum', 
                                 'sm': 'sum', 'avrg_cnwk_sdytrn_rev': 'mean', 
                                 'fyer_salary_totamt_rev2': 'sum',
                                 'jan_salary_am_rev2': 'mean'
                                },
                        values=['rgllbr_co', 'rgllbr_abacpt_labrr_co', 'cnttk_co', 
                                'cnttk_abacpt_labrr_co', 'sm', 'avrg_cnwk_sdytrn_rev',
                                'fyer_salary_totamt_rev2', 'jan_salary_am_rev2'])
data_dart_3_m

avrg_cnwk_sdytrn_rev  cnttk_abacpt_labrr_co  \
corp_code year fo_bbm sexdstn                                                
00100258  2019 가주공장   남                         0.0                      0   
                      여                         0.0                      0   
               서울본부   남                         0.0                      0   
                      여                         0.0                      0   
          2020 서울본부   남                         0.0                      0   
...                                             ...                    ...   
01511558  2021 사무     여                         1.6                      0   
               연구     남                         1.7                      0   
                      여                         0.9                      0   
               임원     남                         1.6                      0   
01513404  2021 공시관리   남                         0.0                      0   

                               cnttk_co  fyer_salary_totamt_rev2  \
corp_code year fo_bbm sexdstn                                      
00100258  2019 가주공장   남               0             5.338710e+05   
                      여               0             1.995360e+05   
               서울본부   남               0             6.440610e+05   
                      여               0             1.671480e+05   
          2020 서울본부   남               0             4.859620e+08   
...                                 ...                      ...   
01511558  2021 사무     여               0             9.570440e+08   
               연구     남               0             1.002211e+09   
                      여               0             1.347675e+09   
               임원     남               0             5.850000e+08   
01513404  2021 공시관리   남               0             0.000000e+00   

                               jan_salary_am_rev2  rgllbr_abacpt_labrr_co  \
corp_code year fo_bbm sexdstn                                               
00100258  2019 가주공장   남                   41067.0                       0   
                      여                   28505.0                       0   
               서울본부   남                   53672.0                       0   
                      여                   33430.0                       0   
          2020 서울본부   남                53996000.0                       0   
...                                           ...                     ...   
01511558  2021 사무     여                95704000.0                       0   
               연구     남               111357000.0                       0   
                      여                89845000.0                       0   
               임원     남               585000000.0                       0   
01513404  2021 공시관리   남                       NaN                       0   

                               rgllbr_co  sm  
corp_code year fo_bbm sexdstn                 
00100258  2019 가주공장   남               13  13  
                      여                7   7  
               서울본부   남               12  12  
                      여                5   5  
          2020 서울본부   남               10  10  
...                                  ...  ..  
01511558  2021 사무     여               10  10  
               연구     남                9   9  
                      여               15  15  
               임원     남                1   1  
01513404  2021 공시관리   남                1   1  

[15480 rows x 8 columns]

In [12]:
# jan_salary_am_rev2(1인 평균 급여)에 오류 너무 많음.. => 새롭게 계산
# 데이터 자체 오류가 많은듯 (단위도 제각각인 것 등등)
data_dart_3_m['jan_salary_am_rev2_new'] = data_dart_3_m.fyer_salary_totamt_rev2 / data_dart_3_m.sm

# 컬럼 재배치
data_dart_3_m = data_dart_3_m.loc[:, ['rgllbr_co', 'rgllbr_abacpt_labrr_co', 'cnttk_co', 
                                'cnttk_abacpt_labrr_co', 'sm', 'avrg_cnwk_sdytrn_rev',
                                'fyer_salary_totamt_rev2', 'jan_salary_am_rev2']]

data_dart_3_m.reset_index(inplace=True)
data_dart_3_m

,corp_code,year,fo_bbm,sexdstn,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2
0,00100258,2019,가주공장,남,13,0,0,0,13,0.0,5.338710e+05,41067.0
1,00100258,2019,가주공장,여,7,0,0,0,7,0.0,1.995360e+05,28505.0
2,00100258,2019,서울본부,남,12,0,0,0,12,0.0,6.440610e+05,53672.0
3,00100258,2019,서울본부,여,5,0,0,0,5,0.0,1.671480e+05,33430.0
4,00100258,2020,서울본부,남,10,0,0,0,10,0.0,4.859620e+08,53996000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15475,01511558,2021,사무,여,10,0,0,0,10,1.6,9.570440e+08,95704000.0
15476,01511558,2021,연구,남,9,0,0,0,9,1.7,1.002211e+09,111357000.0
15477,01511558,2021,연구,여,15,0,0,0,15,0.9,1.347675e+09,89845000.0
15478,01511558,2021,임원,남,1,0,0,0,1,1.6,5.850000e+08,585000000.0


In [13]:
# 기업마다 회계년도, 회계 기한 다 제각각
# 18년도, 19년도, 20년도로 통일 필요
# 종료 년도 활용

data_dart_4['dt_start'] = data_dart_4.dt.str.split(' ~ ').apply(lambda x: x[0][:4])
data_dart_4['dt_end'] = data_dart_4.dt.str.split(' ~ ').apply(lambda x: x[1][:4])
data_dart_4

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,nm,dt,amount,dt_start,dt_end
0,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,103213745,2020,2020
1,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,영업이익,제 19 기,2020.01.01 ~ 2020.12.31,58222516,2020,2020
2,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 19 기,2020.01.01 ~ 2020.12.31,52946039,2020,2020
3,20210316000210,11011,2020,00956028,950130,CFS,연결재무제표,IS,손익계산서,당기순이익,제 19 기,2020.01.01 ~ 2020.12.31,40517092,2020,2020
4,20210316000210,11011,2020,00956028,950130,OFS,재무제표,IS,손익계산서,매출액,제 19 기,2020.01.01 ~ 2020.12.31,92215510,2020,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45725,20210329001000,11011,2020,00140566,002680,OFS,재무제표,IS,손익계산서,당기순이익,제 60 기,2018.01.01 ~ 2018.12.31,-3723580588,2018,2018
45726,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,매출액,제 8 기,2018.01.01 ~ 2018.12.31,276496903,2018,2018
45727,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,영업이익,제 8 기,2018.01.01 ~ 2018.12.31,-3375624884,2018,2018
45728,20210817000495,11011,2020,01265251,291650,OFS,재무제표,IS,손익계산서,법인세차감전 순이익,제 8 기,2018.01.01 ~ 2018.12.31,-7344671390,2018,2018


In [14]:
# merge할 형태로 추가 정제
# 주요계정

data_dart_4_m = data_dart_4.pivot_table(index=['corp_code'],
                        columns=['fs_nm', 'account_nm', 'dt_end'],
                        aggfunc='sum',
                        values='amount').dropna(how='all')
data_dart_4_m.reset_index(inplace=True)
data_dart_4_m

fs_nm      corp_code        연결재무제표                                            \
account_nm                   당기순이익                                       매출액   
dt_end                        2018          2019          2020          2018   
0           00100601  1.708887e+09 -3.594908e+10 -1.287476e+10  8.590537e+10   
1           00100939  3.884233e+10  1.741251e+10  2.506750e+10  3.070208e+11   
2           00101044 -1.215413e+09 -3.466524e+09  1.891757e+10  4.539539e+10   
3           00101220  2.776562e+10  2.193414e+11  1.546643e+11  1.282777e+12   
4           00101433           NaN           NaN           NaN  2.126546e+11   
...              ...           ...           ...           ...           ...   
2144        01489648           NaN           NaN  6.488260e+10           NaN   
2145        01493535           NaN           NaN           NaN           NaN   
2146        01497869           NaN           NaN  3.539817e+11           NaN   
2147        01511558           NaN -1.218023e+07 -2.580928e+07           NaN   
2148        01513404           NaN           NaN           NaN           NaN   

fs_nm                                                               \
account_nm                                법인세차감전 순이익                 
dt_end              2019          2020          2018          2019   
0           4.254962e+10  3.834517e+10  2.522115e+09 -3.509001e+10   
1           3.325394e+11  3.408074e+11  4.037071e+10  2.236389e+10   
2           5.090382e+10  5.331998e+10 -9.524077e+08 -3.057267e+09   
3           2.068395e+12  3.686350e+12  4.987138e+10  2.632410e+11   
4           2.275109e+11  2.482955e+11  1.111417e+10  7.388040e+09   
...                  ...           ...           ...           ...   
2144                 NaN  4.700675e+11           NaN           NaN   
2145                 NaN           NaN           NaN           NaN   
2146                 NaN  9.289882e+10           NaN           NaN   
2147        0.000000e+00  0.000000e+00           NaN -1.217590e+07   
2148                 NaN           NaN           NaN           NaN   

fs_nm                     ...          재무제표                              \
account_nm                ...         당기순이익           매출액                 
dt_end              2020  ...          2020          2018          2019   
0          -1.287476e+10  ... -1.403921e+10  8.590537e+10  4.254962e+10   
1           2.069781e+10  ...  7.352890e+09  2.591157e+11  2.549481e+11   
2           1.664368e+10  ... -9.437470e+08  3.096104e+10  3.360913e+10   
3           2.114264e+11  ... -1.057445e+10  1.672232e+11  1.532798e+11   
4           1.760215e+10  ...  1.045950e+10  1.462167e+11  1.513420e+11   
...                  ...  ...           ...           ...           ...   
2144        8.605996e+10  ...  6.032852e+10           NaN           NaN   
2145                 NaN  ... -2.319289e+07           NaN           NaN   
2146        8.256953e+09  ... -2.440677e+09           NaN           NaN   
2147       -2.578615e+07  ... -2.545989e+07           NaN           NaN   
2148                 NaN  ... -4.048486e+09           NaN           NaN   

fs_nm                                                               \
account_nm                  법인세차감전 순이익                               
dt_end              2020          2018          2019          2020   
0           3.834517e+10  2.790752e+09 -3.290333e+10 -1.403921e+10   
1           2.598999e+11  3.749774e+09  1.281831e+09  3.835301e+09   
2           3.183124e+10 -1.030068e+10 -5.832641e+09 -9.251391e+08   
3           1.434376e+11 -1.558821e+09 -7.176071e+09 -7.978419e+09   
4           1.656553e+11  1.526295e+10  1.023868e+10  1.524329e+10   
...                  ...           ...           ...           ...   
2144        4.534152e+11           NaN           NaN  8.031988e+10   
2145        0.000000e+00           NaN           NaN -2.973448e+07   
2146                 NaN           NaN           NaN -2.263721e+09   
2147   

In [15]:
# dart 데이터 통합
pd.set_option('display.max_columns', 60)

data_dart_m = data_dart_2_m.merge(data_dart_4_m, on='corp_code', how='inner')
data_dart_m = data_dart_m.merge(data_dart_3_m, on='corp_code', how='inner')
data_dart_m

,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,phn_no,fax_no,induty_code,est_dt,crop_cls_kr,"(연결재무제표, 당기순이익, 2018)","(연결재무제표, 당기순이익, 2019)","(연결재무제표, 당기순이익, 2020)","(연결재무제표, 매출액, 2018)","(연결재무제표, 매출액, 2019)","(연결재무제표, 매출액, 2020)","(연결재무제표, 법인세차감전 순이익, 2018)","(연결재무제표, 법인세차감전 순이익, 2019)","(연결재무제표, 법인세차감전 순이익, 2020)","(연결재무제표, 영업이익, 2018)","(연결재무제표, 영업이익, 2019)","(연결재무제표, 영업이익, 2020)","(재무제표, 당기순이익, 2018)","(재무제표, 당기순이익, 2019)","(재무제표, 당기순이익, 2020)","(재무제표, 매출액, 2018)","(재무제표, 매출액, 2019)","(재무제표, 매출액, 2020)","(재무제표, 법인세차감전 순이익, 2018)","(재무제표, 법인세차감전 순이익, 2019)","(재무제표, 법인세차감전 순이익, 2020)","(재무제표, 영업이익, 2018)","(재무제표, 영업이익, 2019)","(재무제표, 영업이익, 2020)",year,fo_bbm,sexdstn,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2
0,00956028,엑세스바이오 인코퍼레이션,"Access Bio, Inc.",엑세스바이오,950130,최영호,K,,0100888833,"65 Clyde Road Suite A Somerset NJ 08873, USA",www.accessbio.net,1-732-873-4040,732-873-4043,21300,20020927,코스닥,-5802229.0,-1.772060e+07,4.051709e+07,36571123.0,3.693964e+07,1.032137e+08,-5580613.0,-1.854018e+07,5.294604e+07,-3982259.0,-6895892.0,58222516.0,735372.0,-6.263868e+06,4.356193e+07,34158734.0,3.206742e+07,9.221551e+07,956988.0,-7.141671e+06,5.597670e+07,870002.0,-2688741.0,58701187.0,2019,사무,남,12,0,0,0,12,3.0,9.874728e+08,82290000.0
1,00956028,엑세스바이오 인코퍼레이션,"Access Bio, Inc.",엑세스바이오,950130,최영호,K,,0100888833,"65 Clyde Road Suite A Somerset NJ 08873, USA",www.accessbio.net,1-732-873-4040,732-873-4043,21300,20020927,코스닥,-5802229.0,-1.772060e+07,4.051709e+07,36571123.0,3.693964e+07,1.032137e+08,-5580613.0,-1.854018e+07,5.294604e+07,-3982259.0,-6895892.0,58222516.0,735372.0,-6.263868e+06,4.356193e+07,34158734.0,3.206742e+07,9.221551e+07,956988.0,-7.141671e+06,5.597670e+07,870002.0,-2688741.0,58701187.0,2019,사무,여,7,0,0,0,7,4.0,4.656252e+08,66518400.0
2,00956028,엑세스바이오 인코퍼레이션,"Access Bio, Inc.",엑세스바이오,950130,최영호,K,,0100888833,"65 Clyde Road Suite A Somerset NJ 08873, USA",www.accessbio.net,1-732-873-4040,732-873-4043,21300,20020927,코스닥,-5802229.0,-1.772060e+07,4.051709e+07,36571123.0,3.693964e+07,1.032137e+08,-5580613.0,-1.854018e+07,5.294604e+07,-3982259.0,-6895892.0,58222516.0,735372.0,-6.263868e+06,4.356193e+07,34158734.0,3.206742e+07,9.221551e+07,956988.0,-7.141671e+06,5.597670e+07,870002.0,-2688741.0,58701187.0,2019,생산,남,17,0,0,0,17,3.0,1.289167e+09,75832800.0
3,00956028,엑세스바이오 인코퍼레이션,"Access Bio, Inc.",엑세스바이오,950130,최영호,K,,0100888833,"65 Clyde Road Suite A Somerset NJ 08873, USA",www.accessbio.net,1-732-873-4040,732-873-4043,21300,20020927,코스닥,-5802229.0,-1.772060e+07,4.051709e+07,36571123.0,3.693964e+07,1.032137e+08,-5580613.0,-1.854018e+07,5.294604e+07,-3982259.0,-6895892.0,58222516.0,735372.0,-6.263868e+06,4.356193e+07,34158734.0,3.206742e+07,9.221551e+07,956988.0,-7.141671e+06,5.597670e+07,870002.0,-2688741.0,58701187.0,2019,생산,여,9,0,0,0,9,5.0,3.961284e+08,44014800.0
4,00956028,엑세스바이오 인코퍼레이션,"Access Bio, Inc.",엑세스바이오,950130,최영호,K,,0100888833,"65 Clyde Road Suite A Somerset NJ 08873, USA",www.accessbio.net,1-732-873-4040,732-873-4043,21300,20020927,코스닥,-5802229.0,-1.772060e+07,4.051709e+07,36571123.0,3.693964e+07,1.032137e+08,-5580613.0,-1.854018e+07,5.294604e+07,-3982259.0,-6895892.0,58222516.0,735372.0,-6.263868e+06,4.356193e+07,34158734.0,3.206742e+07,9.221551e+07,956988.0,-7.141671e+06,5.597670e+07,870002.0,-2688741.0,58701187.0,2019,연구,남,6,0,0,0,6,3.0,4.960836e+08,82681200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13600,01264438,(주)자이언트스텝,GIANTSTEP Inc.,자이언트스텝,289220,"하승봉, 이지철",K,1101113909839,2118813952,서울특별시 강남구 학동로37길 8,www.giantstep.co.kr,02-549-0005,02-549-0903,59113,20080605,코스닥,NaN,-3.142183e+09,-1.422168e+09,NaN,2.052803e+10,2.022311e+10,NaN,-1.779725e+09,-1.459877e+09,NaN,NaN,NaN,NaN,-4.430177e+09,-1.506711e+09,NaN

In [16]:
# 총 1531개 기업
len(data_dart_m.corp_code.unique())

1531

In [17]:
# merge할 형태로 추가 정제
# 국민연금 가입 사업장 내역 + 시도 코드

data_dong_rev['법정동코드2'] = data_dong_rev.법정동코드.apply(lambda x: str(x)[:2])
data_dong_rev = data_dong_rev.set_index('법정동코드2')

dong_dict = data_dong_rev['법정동명'].to_dict()
dong_dict

{'11': '서울특별시',
 '26': '부산광역시',
 '27': '대구광역시',
 '28': '인천광역시',
 '29': '광주광역시',
 '30': '대전광역시',
 '31': '울산광역시',
 '36': '세종특별자치시',
 '41': '경기도',
 '42': '강원도',
 '43': '충청북도',
 '44': '충청남도',
 '45': '전라북도',
 '46': '전라남도',
 '47': '경상북도',
 '48': '경상남도',
 '50': '제주특별자치도'}

In [18]:
# 시도 데이터 결합
data_ins_1['sido'] = data_ins_1.ldongAddrMgplDgCd.map(dong_dict)
data_ins_1

,bzowrRgstNo,dataCrtYm,ldongAddrMgplDgCd,ldongAddrMgplSgguCd,ldongAddrMgplSgguEmdCd,seq,wkplJnngStcd,wkplNm,wkplRoadNmDtlAddr,wkplStylDvcd,sido
0,135813****,202104,41,550,360,32344013,1,나노캠텍(주),경기도 안성시 양성면 양성로,1,경기도
1,653880****,202105,11,440,123,33251682,1,주식회사협진메디텍,서울특별시 마포구 월드컵로,1,서울특별시
2,114871****,202103,11,680,107,31966734,1,(주)프로스테믹스,서울특별시 강남구 압구정로14길,1,서울특별시
3,410814****,202112,29,170,129,36603791,1,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,1,광주광역시
4,107811****,202112,47,111,111,36654621,1,(주)TCC스틸,경상북도 포항시 남구 괴동로,1,경상북도
...,...,...,...,...,...,...,...,...,...,...,...
1981,111843****,202112,11,680,108,36902629,1,미투젠리미티드(영업소),서울특별시 강남구 학동로,1,서울특별시
1982,318810****,201511,43,730,250,20310715,1,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,1,충청북도
1983,433880****,202110,48,310,370,35940676,1,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,1,경상남도
1984,139810****,202112,11,545,101,36501345,1,(주)한창바이오텍,서울특별시 금천구 디지털로9길,1,서울특별시


In [19]:
# 필요 컬럼만 추출
data_ins_1_m = data_ins_1.loc[:, ['seq', 'wkplNm', 'wkplRoadNmDtlAddr', 'sido']]
data_ins_1_m

,seq,wkplNm,wkplRoadNmDtlAddr,sido
0,32344013,나노캠텍(주),경기도 안성시 양성면 양성로,경기도
1,33251682,주식회사협진메디텍,서울특별시 마포구 월드컵로,서울특별시
2,31966734,(주)프로스테믹스,서울특별시 강남구 압구정로14길,서울특별시
3,36603791,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,광주광역시
4,36654621,(주)TCC스틸,경상북도 포항시 남구 괴동로,경상북도
...,...,...,...,...
1981,36902629,미투젠리미티드(영업소),서울특별시 강남구 학동로,서울특별시
1982,20310715,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,충청북도
1983,35940676,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,경상남도
1984,36501345,(주)한창바이오텍,서울특별시 금천구 디지털로9길,서울특별시


In [20]:
# merge할 형태로 추가 정제
# 필요 컬럼만 추출
data_ins_2_m = data_ins_2.loc[:, ['wkplNm', 'vldtVlKrnNm', 'jnngpCnt', 'crrmmNtcAmt']]

# type cast
data_ins_2_m['crrmmNtcAmt'] = data_ins_2_m['crrmmNtcAmt'].astype(float)
data_ins_2_m['jnngpCnt'] = data_ins_2_m['jnngpCnt'].astype(float)

# 연봉 계산
# 인당고지금액
data_ins_2_m['avg_crrmmNtcAmt'] = data_ins_2_m['crrmmNtcAmt'] / data_ins_2_m['jnngpCnt']  

# 임직원 평균 월급 = 인당고지금액 / 9% * 100%
data_ins_2_m['avg_salary_m'] = data_ins_2_m['avg_crrmmNtcAmt'] / 9 * 100

# 임직원 평균 연봉 = 임직원 평균 월급 * 12개월
data_ins_2_m['avg_salary_y'] = data_ins_2_m['avg_salary_m'] * 12
data_ins_2_m

,wkplNm,vldtVlKrnNm,jnngpCnt,crrmmNtcAmt,avg_crrmmNtcAmt,avg_salary_m,avg_salary_y
0,나노캠텍(주),플라스틱 창호 제조업,48.0,14996040.0,312417.500000,3.471306e+06,4.165567e+07
0,주식회사협진메디텍,의료용 기구 소매업,3.0,909980.0,303326.666667,3.370296e+06,4.044356e+07
0,(주)프로스테믹스,사업 및 무형 재산권 중개업,13.0,4803720.0,369516.923077,4.105744e+06,4.926892e+07
0,주식회사 다이나믹디자인,주형 및 금형 제조업,301.0,83023960.0,275827.109635,3.064746e+06,3.677695e+07
0,(주)TCC스틸,그 외 기타 1차 철강 제조업,210.0,95653940.0,455494.952381,5.061055e+06,6.073266e+07
...,...,...,...,...,...,...,...
0,미투젠리미티드(영업소),응용 소프트웨어 개발 및 공급업,3.0,979380.0,326460.000000,3.627333e+06,4.352800e+07
0,(주)아바텍,BIZ_NO미존재사업장,7.0,1089680.0,155668.571429,1.729651e+06,2.075581e+07
0,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,"미장, 타일 및 방수 공사업",3.0,405000.0,135000.000000,1.500000e+06,1.800000e+07
0,(주)한창바이오텍,의약품 도매업,17.0,3945120.0,232065.882353,2.578510e+06,3.094212e+07


In [21]:
# 국민연금 데이터 통합
data_ins_m = data_ins_1_m.merge(data_ins_2_m, on='wkplNm', how='inner')
data_ins_m.drop('seq', axis=1, inplace=True)
data_ins_m = data_ins_m.drop_duplicates()
data_ins_m

,wkplNm,wkplRoadNmDtlAddr,sido,vldtVlKrnNm,jnngpCnt,crrmmNtcAmt,avg_crrmmNtcAmt,avg_salary_m,avg_salary_y
0,나노캠텍(주),경기도 안성시 양성면 양성로,경기도,플라스틱 창호 제조업,48.0,14996040.0,312417.500000,3.471306e+06,4.165567e+07
1,주식회사협진메디텍,서울특별시 마포구 월드컵로,서울특별시,의료용 기구 소매업,3.0,909980.0,303326.666667,3.370296e+06,4.044356e+07
2,(주)프로스테믹스,서울특별시 강남구 압구정로14길,서울특별시,사업 및 무형 재산권 중개업,13.0,4803720.0,369516.923077,4.105744e+06,4.926892e+07
3,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,광주광역시,주형 및 금형 제조업,301.0,83023960.0,275827.109635,3.064746e+06,3.677695e+07
4,(주)TCC스틸,경상북도 포항시 남구 괴동로,경상북도,그 외 기타 1차 철강 제조업,210.0,95653940.0,455494.952381,5.061055e+06,6.073266e+07
...,...,...,...,...,...,...,...,...,...
1987,미투젠리미티드(영업소),서울특별시 강남구 학동로,서울특별시,응용 소프트웨어 개발 및 공급업,3.0,979380.0,326460.000000,3.627333e+06,4.352800e+07
1988,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,충청북도,BIZ_NO미존재사업장,7.0,1089680.0,155668.571429,1.729651e+06,2.075581e+07
1989,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,경상남도,"미장, 타일 및 방수 공사업",3.0,405000.0,135000.000000,1.500000e+06,1.800000e+07
1990,(주)한창바이오텍,서울특별시 금천구 디지털로9길,서울특별시,의약품 도매업,17.0,3945120.0,232065.882353,2.578510e+06,3.094212e+07


In [22]:
# DART & 국민연금 데이터 통합
# key값인 사업자명 데이터 정제

# 공백 제거
pattern_1 = '\s+' 

# 주식회사 문자열 제거
pattern_2 = '주식회사'

# 상용, 일용 뒤 글자까지 제거
pattern_3 = '\(상용|일용\).*'

# 괄호안 문자열 제거 -> 가장 짧은 단어 매칭
pattern_4 = '\(.*?\)'

# 괄호안 문자열 제거 2 -> 가장 짧은 단어 매칭
pattern_5 = '\（.+?\）'

# / 문자 뒤 문자열 제거
pattern_6 = '\/.*'

# - 문자 뒤 문자열 제거
pattern_7 = '\-.*'

def text_preprocess(text):
    text = re.sub(pattern_1, '', text)
    text = re.sub(pattern_2, '', text)
    text = re.sub(pattern_3, '', text)
    text = re.sub(pattern_4, '', text)
    text = re.sub(pattern_5, '', text)
    text = re.sub(pattern_6, '', text)
    text = re.sub(pattern_7, '', text)
    return text

In [23]:
data_dart_m['key'] = data_dart_m.corp_name.apply(text_preprocess)
data_ins_m['key'] = data_ins_m.wkplNm.apply(text_preprocess)

data_dart_m.key, data_ins_m.key

(0        엑세스바이오인코퍼레이션
 1        엑세스바이오인코퍼레이션
 2        엑세스바이오인코퍼레이션
 3        엑세스바이오인코퍼레이션
 4        엑세스바이오인코퍼레이션
              ...     
 13600          자이언트스텝
 13601          자이언트스텝
 13602          자이언트스텝
 13603          자이언트스텝
 13604          자이언트스텝
 Name: key, Length: 13605, dtype: object,
 0                나노캠텍
 1               협진메디텍
 2              프로스테믹스
 3             다이나믹디자인
 4               TCC스틸
             ...      
 1987          미투젠리미티드
 1988              아바텍
 1989            미도이엔씨
 1990           한창바이오텍
 1991    디티알오토모티브울산2공장
 Name: key, Length: 1983, dtype: object)

In [25]:
# key 매칭 dict 생성

match_dict = {}

for name_1 in data_dart_m.key.unique().tolist():
    temp_list = []
    for name_2 in data_ins_m.key.unique().tolist():
        if name_1 in name_2:
            temp_list.append(name_2)
            
    match_dict[name_1] = temp_list
    
    
match_dict

{'엑세스바이오인코퍼레이션': [],
 '글로벌에스엠테크리미티드': [],
 '그레이트리치과기유한공사': [],
 '윙입푸드홀딩스': [],
 '케이만금세기차륜집단유한공사': [],
 '코오롱티슈진': ['코오롱티슈진한국지점'],
 'SBI핀테크솔루션즈': [],
 '영신금속공업': [],
 '웹젠': ['웹젠넥스트'],
 '우리넷': ['우리넷'],
 '마이크로디지탈': ['마이크로디지탈'],
 '새로닉스': ['새로닉스'],
 '로보로보': ['로보로보'],
 '한국유나이티드제약': [],
 '프럼파스트': [],
 '비지에프리테일': [],
 '에이치디씨현대산업개발': [],
 '지니언스': ['지니언스'],
 '시노펙스': ['이스트드림시노펙스'],
 '티웨이홀딩스': ['티웨이홀딩스'],
 '씨앤지하이테크': ['씨앤지하이테크'],
 '삼화왕관': ['삼화왕관'],
 '피엔에이치테크': ['피엔에이치테크'],
 '아모텍': ['아모텍오디가정'],
 '대상홀딩스': ['대상홀딩스'],
 '바이오리더스': ['바이오리더스'],
 '아바코': ['아바코시스템즈리미티드'],
 '미원화학': ['미원화학안양사무소'],
 '팜스빌': ['농업회사법인하이팜스빌'],
 '성도이엔지': ['성도이엔지'],
 '한국맥널티': ['한국맥널티'],
 '심텍홀딩스': ['심텍홀딩스서울지점'],
 '팬엔터테인먼트': ['팬엔터테인먼트'],
 '솔루엠': ['솔루엠헬스케어'],
 '셀트리온헬스케어': ['셀트리온헬스케어홀딩스'],
 '파미셀': ['파미셀'],
 '아이앤씨테크놀로지': [],
 '코미코': ['에코미코'],
 '네오크레마': ['네오크레마'],
 '㈜에스티오': [],
 '텔레칩스': [],
 '에이치알에스': [],
 'KG케미칼': [],
 '엔지스테크널러지': ['엔지스테크널러지'],
 '한국철강': ['한국철강'],
 'KISCO홀딩스': [],
 '㈜핑거': [],
 '메디아나': ['메디아나'],
 '예선테크': ['예선테크'],
 '티에스아이': [

In [26]:
# 1대1 매칭되는 케이스 1123건

t = pd.DataFrame([match_dict]).T.reset_index()
t.columns = ['name1', 'name2']

tt = t[t.name2.str.len() == 1]
tt['name2_r'] = tt.name2.apply(lambda x: x[0])
tt

,name1,name2,name2_r
5,코오롱티슈진,[코오롱티슈진한국지점],코오롱티슈진한국지점
8,웹젠,[웹젠넥스트],웹젠넥스트
9,우리넷,[우리넷],우리넷
10,마이크로디지탈,[마이크로디지탈],마이크로디지탈
11,새로닉스,[새로닉스],새로닉스
...,...,...,...
1523,아래스,[아래스],아래스
1524,오르비텍,[오르비텍],오르비텍
1525,쎄트렉아이,[쎄트렉아이],쎄트렉아이
1529,제이씨케미칼,[에스제이씨케미칼],에스제이씨케미칼


In [27]:
# 중복 매칭 케이스 확인
ttt = pd.DataFrame(tt.name2_r.value_counts())
ttt[ttt.name2_r != 1].index.tolist()

tt[tt.name2_r.isin(ttt[ttt.name2_r != 1].index.tolist())].sort_values(by='name2_r')

,name1,name2,name2_r
1338,DSR제강,[DSR제강],DSR제강
860,DSR,[DSR제강],DSR제강
24,대상홀딩스,[대상홀딩스],대상홀딩스
530,대상,[대상홀딩스],대상홀딩스
557,셀트리온,[셀트리온헬스케어홀딩스],셀트리온헬스케어홀딩스
34,셀트리온헬스케어,[셀트리온헬스케어홀딩스],셀트리온헬스케어홀딩스
252,솔브레인,[솔브레인홀딩스],솔브레인홀딩스
1200,솔브레인홀딩스,[솔브레인홀딩스],솔브레인홀딩스
1208,에이엔피,[에이엔피케이],에이엔피케이
1255,엔피케이,[에이엔피케이],에이엔피케이


In [28]:
# 중복 매칭 케이스 제거
tt = tt[~tt.name1.isin(['DSR', '대상', '셀트리온', '솔브레인', '에이엔피', '엔피케이',
                  '바이오니아', '웰크론', '현대건설', '현대건설기계', '현대코퍼레이션'])]

match_dict_r = tt.set_index('name2_r')['name1'].to_dict()
match_dict_r

{'코오롱티슈진한국지점': '코오롱티슈진',
 '웹젠넥스트': '웹젠',
 '우리넷': '우리넷',
 '마이크로디지탈': '마이크로디지탈',
 '새로닉스': '새로닉스',
 '로보로보': '로보로보',
 '지니언스': '지니언스',
 '이스트드림시노펙스': '시노펙스',
 '티웨이홀딩스': '티웨이홀딩스',
 '씨앤지하이테크': '씨앤지하이테크',
 '삼화왕관': '삼화왕관',
 '피엔에이치테크': '피엔에이치테크',
 '아모텍오디가정': '아모텍',
 '대상홀딩스': '대상홀딩스',
 '바이오리더스': '바이오리더스',
 '아바코시스템즈리미티드': '아바코',
 '미원화학안양사무소': '미원화학',
 '농업회사법인하이팜스빌': '팜스빌',
 '성도이엔지': '성도이엔지',
 '한국맥널티': '한국맥널티',
 '심텍홀딩스서울지점': '심텍홀딩스',
 '팬엔터테인먼트': '팬엔터테인먼트',
 '솔루엠헬스케어': '솔루엠',
 '셀트리온헬스케어홀딩스': '셀트리온헬스케어',
 '파미셀': '파미셀',
 '에코미코': '코미코',
 '네오크레마': '네오크레마',
 '엔지스테크널러지': '엔지스테크널러지',
 '한국철강': '한국철강',
 '메디아나': '메디아나',
 '예선테크': '예선테크',
 '티에스아이시스템': '티에스아이',
 '롯데지주': '롯데지주',
 '위세아이텍': '위세아이텍',
 'CJ씨푸드이천공장': 'CJ씨푸드',
 '올릭스글로벌': '올릭스',
 '쿠쿠홀딩스': '쿠쿠홀딩스',
 '피씨엘오': '피씨엘',
 '모두락애경산업': '애경산업',
 '휴네시온': '휴네시온',
 '명승파워넷': '파워넷',
 '조선선재': '조선선재',
 '대경스마트파트론': '파트론',
 '박셀바이오': '박셀바이오',
 '휴니드테크놀러지스': '휴니드테크놀러지스',
 '덕신하우징': '덕신하우징',
 '신풍제약': '신풍제약',
 '화산에스티아이': '에스티아이',
 '피지에스이스트아시아피티이엘티디': '지에스이',
 '유니크피스': '유니크',
 '아이큐어

In [29]:
# matching key setting
data_ins_m['key2'] = data_ins_m['key'].map(match_dict_r)
data_ins_m.drop('key', axis=1, inplace=True)
data_ins_m

,wkplNm,wkplRoadNmDtlAddr,sido,vldtVlKrnNm,jnngpCnt,crrmmNtcAmt,avg_crrmmNtcAmt,avg_salary_m,avg_salary_y,key2
0,나노캠텍(주),경기도 안성시 양성면 양성로,경기도,플라스틱 창호 제조업,48.0,14996040.0,312417.500000,3.471306e+06,4.165567e+07,나노캠텍
1,주식회사협진메디텍,서울특별시 마포구 월드컵로,서울특별시,의료용 기구 소매업,3.0,909980.0,303326.666667,3.370296e+06,4.044356e+07,협진
2,(주)프로스테믹스,서울특별시 강남구 압구정로14길,서울특별시,사업 및 무형 재산권 중개업,13.0,4803720.0,369516.923077,4.105744e+06,4.926892e+07,NaN
3,주식회사 다이나믹디자인,광주광역시 북구 첨단연신로29번길,광주광역시,주형 및 금형 제조업,301.0,83023960.0,275827.109635,3.064746e+06,3.677695e+07,다이나믹디자인
4,(주)TCC스틸,경상북도 포항시 남구 괴동로,경상북도,그 외 기타 1차 철강 제조업,210.0,95653940.0,455494.952381,5.061055e+06,6.073266e+07,TCC스틸
...,...,...,...,...,...,...,...,...,...,...
1987,미투젠리미티드(영업소),서울특별시 강남구 학동로,서울특별시,응용 소프트웨어 개발 및 공급업,3.0,979380.0,326460.000000,3.627333e+06,4.352800e+07,NaN
1988,(주)아바텍,충청북도 옥천군 옥천읍 의료단지길,충청북도,BIZ_NO미존재사업장,7.0,1089680.0,155668.571429,1.729651e+06,2.075581e+07,NaN
1989,(주)미도이엔씨-(일용)DL건설/거제00대대이전 및 양여부지 개발사업 중 방수공사,경상남도 거제시 연초면 송정이목로,경상남도,"미장, 타일 및 방수 공사업",3.0,405000.0,135000.000000,1.500000e+06,1.800000e+07,NaN
1990,(주)한창바이오텍,서울특별시 금천구 디지털로9길,서울특별시,의약품 도매업,17.0,3945120.0,232065.882353,2.578510e+06,3.094212e+07,한창바이오텍


In [30]:
# merge
data_m = data_dart_m.merge(data_ins_m, left_on='key', right_on='key2', how='inner')
data_m

,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,phn_no,fax_no,induty_code,est_dt,crop_cls_kr,"(연결재무제표, 당기순이익, 2018)","(연결재무제표, 당기순이익, 2019)","(연결재무제표, 당기순이익, 2020)","(연결재무제표, 매출액, 2018)","(연결재무제표, 매출액, 2019)","(연결재무제표, 매출액, 2020)","(연결재무제표, 법인세차감전 순이익, 2018)","(연결재무제표, 법인세차감전 순이익, 2019)","(연결재무제표, 법인세차감전 순이익, 2020)","(연결재무제표, 영업이익, 2018)","(연결재무제표, 영업이익, 2019)","(연결재무제표, 영업이익, 2020)","(재무제표, 당기순이익, 2018)","(재무제표, 당기순이익, 2019)",...,"(재무제표, 매출액, 2019)","(재무제표, 매출액, 2020)","(재무제표, 법인세차감전 순이익, 2018)","(재무제표, 법인세차감전 순이익, 2019)","(재무제표, 법인세차감전 순이익, 2020)","(재무제표, 영업이익, 2018)","(재무제표, 영업이익, 2019)","(재무제표, 영업이익, 2020)",year,fo_bbm,sexdstn,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,key,wkplNm,wkplRoadNmDtlAddr,sido,vldtVlKrnNm,jnngpCnt,crrmmNtcAmt,avg_crrmmNtcAmt,avg_salary_m,avg_salary_y,key2
0,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,경영지원,남,38,0,0,0,38,1.866667,4.126733e+09,1.098570e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
1,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,경영지원,여,15,0,0,0,15,3.100000,1.247388e+09,8.617767e+07,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
2,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,바이오,남,26,0,0,0,26,2.600000,5.559941e+09,2.174105e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
3,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,바이오,여,20,0,0,0,20,3.100000,2.921860e+09,1.474415e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
4,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,바이오사업부,남,15,0,0,0,15,2.000000,2.133600e+09,1.428000e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9610,01264438,(주)자이언트스텝,GIANTSTEP Inc.,자이언트스텝,289220,"하승봉, 이지철",K,1101113909839,2118813952,서울특별시 강남구 학동로37길 8,www.giantstep.co.kr,02-549-0005,0

In [31]:
# 중복 확인. 제거.

print(len(data_m.corp_name.unique()))
print(len(data_m.wkplNm.unique()))

1112
1117


In [32]:
data_m[['corp_name', 'wkplNm']].drop_duplicates().corp_name.value_counts()[:5].index

Index(['주식회사 파라텍', '(주)시공테크', '(주)보광산업', '(주)국보', '계룡건설산업(주)'], dtype='object')

In [33]:
data_m = data_m.loc[~data_m.corp_name.isin(['계룡건설산업(주)', '(주)국보', '주식회사 파라텍', '(주)시공테크', '(주)보광산업'])]
data_m

,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,phn_no,fax_no,induty_code,est_dt,crop_cls_kr,"(연결재무제표, 당기순이익, 2018)","(연결재무제표, 당기순이익, 2019)","(연결재무제표, 당기순이익, 2020)","(연결재무제표, 매출액, 2018)","(연결재무제표, 매출액, 2019)","(연결재무제표, 매출액, 2020)","(연결재무제표, 법인세차감전 순이익, 2018)","(연결재무제표, 법인세차감전 순이익, 2019)","(연결재무제표, 법인세차감전 순이익, 2020)","(연결재무제표, 영업이익, 2018)","(연결재무제표, 영업이익, 2019)","(연결재무제표, 영업이익, 2020)","(재무제표, 당기순이익, 2018)","(재무제표, 당기순이익, 2019)",...,"(재무제표, 매출액, 2019)","(재무제표, 매출액, 2020)","(재무제표, 법인세차감전 순이익, 2018)","(재무제표, 법인세차감전 순이익, 2019)","(재무제표, 법인세차감전 순이익, 2020)","(재무제표, 영업이익, 2018)","(재무제표, 영업이익, 2019)","(재무제표, 영업이익, 2020)",year,fo_bbm,sexdstn,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn_rev,fyer_salary_totamt_rev2,jan_salary_am_rev2,key,wkplNm,wkplRoadNmDtlAddr,sido,vldtVlKrnNm,jnngpCnt,crrmmNtcAmt,avg_crrmmNtcAmt,avg_salary_m,avg_salary_y,key2
0,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,경영지원,남,38,0,0,0,38,1.866667,4.126733e+09,1.098570e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
1,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,경영지원,여,15,0,0,0,15,3.100000,1.247388e+09,8.617767e+07,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
2,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,바이오,남,26,0,0,0,26,2.600000,5.559941e+09,2.174105e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
3,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,바이오,여,20,0,0,0,20,3.100000,2.921860e+09,1.474415e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
4,01245062,코오롱티슈진(주),"Kolon TissueGene, Inc.",코오롱티슈진,950160,"노문종, 한성수",K,,20174063267,"Rockville, MD, USA 9713 Key West Ave Suite 300",tissuegene.com,1-301-921-6000,,21102,19990609,코스닥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29324152.0,-4.288476e+07,...,2.603554e+06,3.013298e+06,-29294662.0,-4.284035e+07,-3.530762e+07,-29937807.0,-38757201.0,-35215717.0,2021,바이오사업부,남,15,0,0,0,15,2.000000,2.133600e+09,1.428000e+08,코오롱티슈진,코오롱티슈진(주)한국지점,경기도 과천시 코오롱로,경기도,의료 기기 도매업,14.0,4986240.0,356160.000000,3.957333e+06,4.748800e+07,코오롱티슈진
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9610,01264438,(주)자이언트스텝,GIANTSTEP Inc.,자이언트스텝,289220,"하승봉, 이지철",K,1101113909839,2118813952,서울특별시 강남구 학동로37길 8,www.giantstep.co.kr,02-549-0005,0

In [34]:
# 최종 1107개 기업

print(len(data_m.corp_name.unique()))
print(len(data_m.wkplNm.unique()))

1107
1107


In [35]:
data_m.to_excel('data_m.xlsx', index=False, encoding='cp949')